In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
import IPython
import subprocess
import time

In [ ]:
endpointURL_reactome_v65 = "http://localhost:3030/reactome_v65/query"
endpointURL_reactome_pc = "http://localhost:3030/reactome_pc/query"
endpointURL_panther_pc = "http://localhost:3030/panther_pc/query"
endpointURL_pathbank_pc = "http://localhost:3030/pathbank_pc/query"
endpointURL_humancyc_pc = "http://localhost:3030/humancyc_pc/query"
endpointURL_kegg_pc = "http://localhost:3030/kegg_pc/query"
endpointURL_pid_pc = "http://localhost:3030/pid_pc/query"
endpointURL_inoh_pc = "http://localhost:3030/inoh_pc/query"
endpointURL_netpath_pc = "http://localhost:3030/netpath_pc/query"
rdfFormat = "turtle"

In [ ]:
reactomeVersion = 65 
prefixes = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>

PREFIX chebi: <http://purl.obolibrary.org/obo/chebi/>
PREFIX chebidb: <http://purl.obolibrary.org/obo/CHEBI_>
PREFIX chebirel: <http://purl.obolibrary.org/obo/CHEBI#>
PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>

PREFIX bp3: <http://www.biopax.org/release/biopax-level3.owl#>

# Homo_sapiens-20170221.owl
#PREFIX reactome: <http://www.reactome.org/biopax/59/48887#> 
#
# Homo_sapiens-20210608.owl
#PREFIX reactome: <http://www.reactome.org/biopax/77/48887#>
#
# Homo_sapiens-20220614.owl
#PREFIX reactome: <http://www.reactome.org/biopax/81/48887#>
#
# Homo_sapiens-20221130.owl
#PREFIX reactome: <http://www.reactome.org/biopax/83/48887#>

PREFIX reactome: <http://www.reactome.org/biopax/{}/48887#>
""".format(reactomeVersion)

biopaxURI = "http://www.biopax.org/release/biopax-level3.owl#"

In [ ]:
def displaySparqlResults(results):
    '''
    Displays as HTML the result of a SPARQLWrapper query in a Jupyter notebook.
    
        Parameters:
            results (dictionnary): the result of a call to SPARQLWrapper.query().convert()
    '''
    variableNames = results['head']['vars']
    tableCode = '<table><tr><th>{}</th></tr><tr>{}</tr></table>'.format('</th><th>'.join(variableNames), '</tr><tr>'.join('<td>{}</td>'.format('</td><td>'.join([row[vName]['value'] if vName in row.keys() else "&nbsp;" for vName in variableNames]))for row in results["results"]["bindings"]))
    IPython.display.display(IPython.display.HTML(tableCode))

## Mappings to UniProt

In [ ]:
# First SPARQL query using the string "UniProt" to map to UniProt resources
query1 = """
SELECT ?metric ?count
# Number of instances of Protein
WHERE {
OPTIONAL {
    {
        SELECT ?metric (COUNT(DISTINCT ?protein) AS ?count)
        WHERE {
            BIND("Protein" AS ?metric)
            ?protein rdf:type/(rdfs:subClassOf*) bp3:Protein .
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_ref) AS ?count)
        WHERE {
            BIND("ProteinReference" AS ?metric)
            ?protein_ref rdf:type bp3:ProteinReference .
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_without_ref) AS ?count)
        WHERE {
            BIND("Protein with no PR" AS ?metric)
            OPTIONAL {
            ?protein_without_ref rdf:type/(rdfs:subClassOf*) bp3:Protein .
            FILTER NOT EXISTS {
                ?protein_without_ref bp3:entityReference ?ref .
                ?ref rdf:type bp3:ProteinReference .
            }
        }
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_that_map_to_uniprot) AS ?count)
        WHERE {
            BIND("Uniprot mappings" AS ?metric)
            ?protein_that_map_to_uniprot rdf:type bp3:Protein .
            ?protein_that_map_to_uniprot bp3:entityReference ?protein_ref .
            ?protein_ref rdf:type bp3:ProteinReference .
            ?protein_ref bp3:xref ?protein_ref_xref .
            ?protein_ref_xref rdf:type bp3:UnificationXref .
            ?protein_ref_xref bp3:db "UniProt" .
            ?protein_ref_xref bp3:id ?up_id .
        }
        GROUP BY ?metric
    }
    UNION 
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_no_map_to_uniprot) AS ?count)
        WHERE {
            BIND("Protein with no mapping" AS ?metric)
            ?protein_no_map_to_uniprot rdf:type/(rdfs:subClassOf*) bp3:Protein .
            ?protein_no_map_to_uniprot bp3:entityReference ?protein_ref .
            FILTER NOT EXISTS {
                ?protein_ref bp3:xref ?xref .
                ?xref rdf:type bp3:UnificationXref .
                ?xref bp3:db "UniProt" .
                ?xref bp3:id ?id .
            }
        }
        GROUP BY ?metric
    }
    UNION 
    {
        SELECT ?metric (COUNT(DISTINCT ?up_id) AS ?count)
        WHERE {
            BIND("Unique Uniprot" AS ?metric)
            ?unif_xref rdf:type bp3:UnificationXref .
            ?unif_xref bp3:db "UniProt" .
            ?unif_xref bp3:id ?up_id .
        }
        GROUP BY ?metric
    }
   }
}
ORDER BY ?metric

"""

# Second SPARQL query using the string "uniprot knowledgebase" to map to UniProt resources
query2 = """
SELECT ?metric ?count
# Number of instances of Protein
WHERE {
    {
        SELECT ?metric (COUNT(DISTINCT ?protein) AS ?count)
        WHERE {
            BIND("Protein" AS ?metric)
            ?protein rdf:type/(rdfs:subClassOf*) bp3:Protein .
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_ref) AS ?count)
        WHERE {
            BIND("ProteinReference" AS ?metric)
            ?protein_ref rdf:type bp3:ProteinReference .
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_without_ref) AS ?count)
        WHERE {
      BIND("Protein with no PR" AS ?metric)
      OPTIONAL {
            ?protein_without_ref rdf:type/(rdfs:subClassOf*) bp3:Protein .
            FILTER NOT EXISTS {
                ?protein_without_ref bp3:entityReference ?ref .
                ?ref rdf:type bp3:ProteinReference .
            }
        }
    }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_that_map_to_uniprot) AS ?count)
        WHERE {
            BIND("Uniprot mappings" AS ?metric)
            ?protein_that_map_to_uniprot rdf:type bp3:Protein .
            ?protein_that_map_to_uniprot bp3:entityReference ?protein_ref .
            ?protein_ref rdf:type bp3:ProteinReference .
            ?protein_ref bp3:xref ?protein_ref_xref .
            ?protein_ref_xref rdf:type bp3:UnificationXref .
            ?protein_ref_xref bp3:db "uniprot knowledgebase" .
            ?protein_ref_xref bp3:id ?up_id .
        }
        GROUP BY ?metric
    }
    UNION 
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_no_map_to_uniprot) AS ?count)
        WHERE {
            BIND("Protein with no mapping" AS ?metric)
            ?protein_no_map_to_uniprot rdf:type/(rdfs:subClassOf*) bp3:Protein .
            ?protein_no_map_to_uniprot bp3:entityReference ?protein_ref .
            FILTER NOT EXISTS {
                ?protein_ref bp3:xref ?xref .
                ?xref rdf:type bp3:UnificationXref .
                ?xref bp3:db "uniprot knowledgebase" .
                ?xref bp3:id ?id .
            }
        }
        GROUP BY ?metric
    }
    UNION 
    {
        SELECT ?metric (COUNT(DISTINCT ?up_id) AS ?count)
        WHERE {
            BIND("Unique Uniprot" AS ?metric)
            ?unif_xref rdf:type bp3:UnificationXref .
            ?unif_xref bp3:db "uniprot knowledgebase" .
            ?unif_xref bp3:id ?up_id .
        }
        GROUP BY ?metric
    }
}
ORDER BY ?metric

"""

TODO: change the path to fuseki server and BioPAX files + BioPAX ontology in the command line to launch the SPARQL endpoint

#### Query on Reactome BioPAX standalone export version 65

In [ ]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/ReactomeBioPAX/_00_Reactome_Data_v65/Homo_sapiens.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/reactome_v65']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_reactome_v65)
sparql.setQuery(prefixes+query1)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open(f"../Results/reactome_v65_mappings_uniprot.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

#### Query on BioPAX export of Reactome from PathwayCommons

In [ ]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.reactome.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/reactome_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_reactome_pc)
sparql.setQuery(prefixes+query2)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open(f"../Results/reactome_pc_mappings_uniprot.csv", "wb") as f:
    f.write(results)

# end process    
process.kill()
time.sleep(60)

#### Query on BioPAX export of Panther from PathwayCommons

In [ ]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.panther.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/panther_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_panther_pc)
sparql.setQuery(prefixes+query2)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/panther_pc_mappings_uniprot.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

#### Query on BioPAX export of PathBank from PathwayCommons

In [ ]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.pathbank.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/pathbank_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_pathbank_pc)
sparql.setQuery(prefixes+query2)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/pathbank_pc_mappings_uniprot.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

#### Query on BioPAX export of HumanCyc from PathwayCommons

In [ ]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.humancyc.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/humancyc_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_humancyc_pc)
sparql.setQuery(prefixes+query2)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/humancyc_pc_mappings_uniprot.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

#### Query on BioPAX export of KEGG from PathwayCommons

In [ ]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.kegg.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/kegg_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_kegg_pc)
sparql.setQuery(prefixes+query2)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/kegg_pc_mappings_uniprot.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

#### Query on BioPAX export of PID from PathwayCommons

In [ ]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.pid.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/pid_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_pid_pc)
sparql.setQuery(prefixes+query2)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/pid_pc_mappings_uniprot.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

#### Query on BioPAX export of INOH from PathwayCommons

In [ ]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.inoh.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/inoh_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_inoh_pc)
sparql.setQuery(prefixes+query2)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/inoh_pc_mappings_uniprot.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

#### Query on BioPAX export of NetPath from PathwayCommons

In [ ]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.netpath.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/netpath_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_netpath_pc)
sparql.setQuery(prefixes+query2)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/netpath_pc_mappings_uniprot.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)